# Programmatically define methods:

In [ ]:
struct MyNumber
    x::Float64
end

for op = (:sin, :cos, :tan, :log, :exp)
    eval(quote
        Base.$op(a::MyNumber) = MyNumber($op(a.x))
    end)
end

# Custom string literals

In [1]:
struct KeepZerosFloat{T<:AbstractFloat}
    x::T
    n::Int64 # extra tail of n zeros
end

In [2]:
KeepZerosFloat(z::T) where {T<:AbstractFloat} = KeepZerosFloat{T}(z, 0)

KeepZerosFloat

In [3]:
KeepZerosFloat(3.0)

KeepZerosFloat{Float64}(3.0, 0)

In [57]:
Base.show(io::IO, z::KeepZerosFloat) = print(io, string(z.x) * join(fill('0', z.n)))

In [58]:
KeepZerosFloat(3.0)

3.0

In [59]:
KeepZerosFloat(3.0000)

3.0

In [77]:
macro k_str(x)
    xF64 = parse(Float64, x)
    s = string(xF64)
    zeros_str = replace(x, s => "")
    @assert zeros_str === join(fill('0', length(zeros_str)))
    
    KeepZerosFloat{Float64}(xF64, length(zeros_str))
end

@k_str (macro with 1 method)

In [80]:
k"3.0"

3.0

In [81]:
k"3.000"

3.000

In [84]:
x = k"3.1410000000"

3.1410000000

In [85]:
typeof(x)

KeepZerosFloat{Float64}

# Example: Unroll.jl

In [182]:
] dev https://github.com/StephenVavasis/Unroll.jl

   Cloning git-repo `https://github.com/StephenVavasis/Unroll.jl`
Fetching: [========================================>]  100.0 %.0 % %  Updating git-repo `https://github.com/StephenVavasis/Unroll.jl`
 Resolving package versions...


┌ Info: Assigning UUID bec05c2e-cc06-5df2-90c3-340a513e71ff to Unroll
└ @ Pkg.Types C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Pkg\src\Types.jl:841


  Updating `C:\Users\carsten\Desktop\julia-workshop-2019\Project.toml`
  [bec05c2e] + Unroll v0.0.0 [`C:\Users\carsten\.julia\dev\Unroll`]
  Updating `C:\Users\carsten\Desktop\julia-workshop-2019\Manifest.toml`
  [bec05c2e] + Unroll v0.0.0 [`C:\Users\carsten\.julia\dev\Unroll`]


In [1]:
using Revise

In [2]:
using Unroll, BenchmarkTools

┌ Info: Recompiling stale cache file C:\Users\carsten\.julia\compiled\v1.1\Unroll\GnJ37.ji for Unroll [bec05c2e-cc06-5df2-90c3-340a513e71ff]
└ @ Base loading.jl:1184


In [34]:
function f()
    z = 1.0
    for i in 1:10
        z += i*rand()
    end
    z
end

f (generic function with 1 method)

In [35]:
@time f()

  0.012787 seconds (32.05 k allocations: 1.766 MiB)


30.116107186413537

In [36]:
@code_lowered f()

CodeInfo(
1 ─       z = 1.0
│   %2  = 1:10
│         #temp# = (Base.iterate)(%2)
│   %4  = #temp# === nothing
│   %5  = (Base.not_int)(%4)
└──       goto #4 if not %5
2 ┄ %7  = #temp#
│         i = (Core.getfield)(%7, 1)
│   %9  = (Core.getfield)(%7, 2)
│   %10 = z
│   %11 = i
│   %12 = (Main.rand)()
│   %13 = %11 * %12
│         z = %10 + %13
│         #temp# = (Base.iterate)(%2, %9)
│   %16 = #temp# === nothing
│   %17 = (Base.not_int)(%16)
└──       goto #4 if not %17
3 ─       goto #2
4 ┄       return z
)

In [37]:
@btime f()

  21.642 ns (0 allocations: 0 bytes)


38.09206697201813

In [38]:
function g()
    z = 1.0
    @unroll for i in 1:10
        z += i*rand()
    end 
    z
end

g (generic function with 1 method)

In [39]:
@time g()

  0.029493 seconds (106.86 k allocations: 5.611 MiB)


41.343279050125766

In [40]:
@btime g();

  18.218 ns (0 allocations: 0 bytes)


In [41]:
@code_lowered g()

CodeInfo(
1 ─       z = 1.0
│   %2  = z
│   %3  = (Main.rand)()
│   %4  = 1 * %3
│         z = %2 + %4
│   %6  = z
│   %7  = (Main.rand)()
│   %8  = 2 * %7
│         z = %6 + %8
│   %10 = z
│   %11 = (Main.rand)()
│   %12 = 3 * %11
│         z = %10 + %12
│   %14 = z
│   %15 = (Main.rand)()
│   %16 = 4 * %15
│         z = %14 + %16
│   %18 = z
│   %19 = (Main.rand)()
│   %20 = 5 * %19
│         z = %18 + %20
│   %22 = z
│   %23 = (Main.rand)()
│   %24 = 6 * %23
│         z = %22 + %24
│   %26 = z
│   %27 = (Main.rand)()
│   %28 = 7 * %27
│         z = %26 + %28
│   %30 = z
│   %31 = (Main.rand)()
│   %32 = 8 * %31
│         z = %30 + %32
│   %34 = z
│   %35 = (Main.rand)()
│   %36 = 9 * %35
│         z = %34 + %36
│   %38 = z
│   %39 = (Main.rand)()
│   %40 = 10 * %39
│         z = %38 + %40
└──       return z
)